In [43]:
import cv2
import numpy as np
from queue import Queue
from threading import Thread
import time
import matplotlib.pyplot as plt

In [44]:
INPUT_VIDEO_PATH = "data/videos/Rec16-1_trimmed.mp4"
OUTPUT_VIDEO_PATH = "output/videos/Rec16-1_trimmed_output.mp4"

In [45]:
START_X = 80  # X-coordinate of the top-left corner of the cropped region
START_Y = 50  # Y-coordinate of the top-left corner of the cropped region
CROP_WIDTH = 1380  # Width of the cropped region
CROP_HEIGHT = 780  # Height of the cropped region

In [46]:
def read_frames(cap, frame_queue, max_queue_size):
    while True:
        if frame_queue.qsize() < max_queue_size:
            ret, frame = cap.read()
            if not ret:
                break
            frame_queue.put(frame)
        else:
            time.sleep(0.1)
    frame_queue.put(None)


In [47]:
def read_frames(cap, frame_queue, max_queue_size):
    while True:
        if frame_queue.qsize() < max_queue_size:
            ret, frame = cap.read()
            if not ret:
                break
            frame_queue.put(frame)
        else:
            time.sleep(0.1)
    frame_queue.put(None)

In [48]:
def crop_frames(frame_queue, result_queue):
    while True:
        frame = frame_queue.get()
        if frame is None:
            break
        
        # Get the dimensions of the original frame
        height, width = frame.shape[:2]
        
        # Calculate the coordinates for the top right corner
        start_x = START_X
        start_y = START_Y
        
        # Crop the frame
        cropped_frame = frame[start_y:start_y+CROP_HEIGHT, start_x:start_x+CROP_WIDTH]
        
        result_queue.put(cropped_frame)
    result_queue.put(None)

In [49]:
def write_video(result_queue, out):
    while True:
        frame = result_queue.get()
        if frame is None:
            break
        out.write(frame)

In [50]:
def process_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (CROP_WIDTH, CROP_HEIGHT))

    frame_queue = Queue(maxsize=30)
    result_queue = Queue(maxsize=30)
    
    read_thread = Thread(target=read_frames, args=(cap, frame_queue, 30))
    crop_thread = Thread(target=crop_frames, args=(frame_queue, result_queue))
    write_thread = Thread(target=write_video, args=(result_queue, out))
    
    read_thread.start()
    crop_thread.start()
    write_thread.start()

    read_thread.join()
    crop_thread.join()
    write_thread.join()

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [51]:
def crop_and_show_first_frame(video_path, crop_width=320, crop_height=240):
    """
    Read the first frame of a video, crop the top right corner, and display it.
    
    Args:
    video_path (str): Path to the input video file
    crop_width (int): Width of the cropped region (default: 320)
    crop_height (int): Height of the cropped region (default: 240)
    """
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Read the first frame
    ret, frame = cap.read()
    
    if not ret:
        print("Failed to read the video file.")
        return
    
    # Get the dimensions of the original frame
    height, width = frame.shape[:2]
    
    # Calculate the coordinates for the top right corner
    start_x = 80
    start_y = 50
    
    # Crop the frame
    cropped_frame = frame[start_y:start_y+crop_height, start_x:start_x+crop_width]

    
    # Convert BGR to RGB (OpenCV uses BGR by default)
    rgb_frame = cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2RGB)
    
    # Display the cropped frame
    plt.imshow(rgb_frame)
    plt.title("Cropped First Frame")
    plt.axis('off')
    plt.show()
    
    # Release the video capture object
    cap.release()

# crop_and_show_first_frame(INPUT_VIDEO_PATH, crop_width=1380, crop_height=780)

In [52]:
process_video(INPUT_VIDEO_PATH, OUTPUT_VIDEO_PATH)